In [1]:
# !pip install langchain
# !pip install transformers
# !pip install faiss-cpu
# !pip install chromadb
# !pip install rouge-score
# !pip install streamlit fastapi
# ! pip install -U langchain-community
# !pip install sentence-transformers
# !pip install openai==0.28

In [6]:
# Imports
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter
from bs4 import BeautifulSoup
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms.base import LLM
from rouge_score import rouge_scorer
from langchain.cache import InMemoryCache
import streamlit as st
import openai
from typing import Optional, List

# Corpus Preparation
- **Corpus Preparation**: We obtained a domain-specific corpus from a sample URL (a quantum computing blog) and preprocessed it by cleaning the HTML tags.
- **Text Splitting**: The text was split into smaller chunks using Langchain’s `RecursiveCharacterTextSplitter` with a chunk size of 500 characters and a 100-character overlap to ensure context continuity between chunks.

In [7]:
# Text cleaning
def fetch_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Remove unwanted HTML tags
    for tag in soup(["script", "style"]):
        tag.decompose()

    # Get text content
    text = ' '.join([p.get_text() for p in soup.find_all('p')])
    return text

# URL input
url = "https://medium.com/@vignesh2659/quantum-computing-abd85aa5da9d"
corpus = fetch_text_from_url(url)

# Split the text into smaller chunks using Langchain's TextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
corpus_chunks = splitter.split_text(corpus)

In [8]:
# Displaying a sample chunk
corpus_chunks[:2]

['Sign up Sign in Sign up Sign in Vignesh R Follow -- Listen Share Quantum Realm in the MARVEL multiverse, our very own Ant-man has one of the coolest superpowers out there. He becomes either too tiny or too huge. This fancy term is indeed nice to talk about. Introduction Likewise, back to our topic of interest, there comes Quantum computing. Quantum computers are powerful than supercomputers and are present in Google, IBM, and Rigetti. A fancy term here is Quantum Supremacy. Google had achieved',
 'present in Google, IBM, and Rigetti. A fancy term here is Quantum Supremacy. Google had achieved Quantum Supremacy with its Quantum computer in 2019, Sycamore. It can perform a calculation in mere seconds which might take the world’s fastest supercomputer around thousands of years. That is the level the world has gone into. Sycamore is a 53-qubit computer. Such computers need to be kept under conditions with a temperature close to absolute zero. Quantum Physics Quantum computing falls under

# Vectorize the Corpus
- **Embedding Generation**: We used `HuggingFaceEmbeddings` to create embeddings from the pre-trained model `sentence-transformers/all-mpnet-base-v2`. This allows us to convert each chunk of the corpus into a numerical vector that captures its semantic meaning.
- **Vector Store**: The embeddings were stored in a FAISS vector store, which is an efficient structure for storing and retrieving vectors based on similarity search.


In [9]:
# Initialize HuggingFace embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Create embeddings for the chunks
chunk_embeddings = embedding_model.embed_documents(corpus_chunks)

# Initialize FAISS vector store and add texts with their embeddings
vector_store = FAISS.from_texts(texts=corpus_chunks, embedding=embedding_model)

C:\Users\r.vignesh\AppData\Local\Temp\ipykernel_8836\1689185818.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
C:\Users\r.vignesh\Anaconda3\envs\rag_env_bytesized\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# Implement the Retrieval Component
- **Retriever Setup**: We initialized a FAISS-based retriever from the stored embeddings. The retriever efficiently finds the top relevant documents for any input query by comparing the similarity between the query and document vectors.


In [10]:
# Retriever using FAISS
retriever = vector_store.as_retriever()

# Example
query = "What is Superposition?"
retrieved_docs = retriever.get_relevant_documents(query)

# Display retrieved results
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i + 1}: {doc.page_content[:200]}...\n")

Document 1: with a temperature close to absolute zero. Quantum Physics Quantum computing falls under a study called Quantum Physics. Quantum computing’s heart and soul resides in what we call as Qubits (Quantum b...

Document 2: either a head or a tail. However, during the spin, there are 2 possibilities, both head and a tail. That position of a coin having both at the same time is what a qubit is. This state is called Superp...

Document 3: answers cancel each other out. Hence this way, amplitude with the right answer remains as the only possible outcome. Quantum computers function using a process called superconductivity. We have a chip...

Document 4: technical aspect of it. In Quantum Mechanics, the following is as explained by Scott Aaronson, who is a Quantum scientist from the University of Texas in Austin. Amplitude, an amplitude of a positive ...



C:\Users\r.vignesh\AppData\Local\Temp\ipykernel_8836\718014876.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


# Build the Generation Component
- **Generation with GPT-4o-mini**: We created a function to combine the retrieved document chunks and pass them as context to OpenAI's GPT-4o-mini model, which generates a coherent response based on the query and context.


In [11]:
# Generate a response using the GPT-4o-mini model
def generate_response(query, retrieved_docs): 
    context = " ".join([doc.page_content for doc in retrieved_docs]) # Combine into a single context
    
    # Using OpenAI's API
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Context: {context}"},
            {"role": "user", "content": query}
        ],
        max_tokens=300,
        temperature=0.7
    )

    # Extracting the first choice's content from the response
    return response['choices'][0]['message']['content']

# Combine the Retrieval and Generation Components
- **Retrieval-Augmented Generation**: We used Langchain’s `RetrievalQA` to combine the retrieval and generation steps into one integrated system. This ensures that the language model generates responses based on the most relevant retrieved documents.


In [12]:
# Custom wrapper for Openai function
class GptLLM(LLM):
    def _call(self, prompt: str, retrieved_docs: List = [], stop: Optional[List[str]] = None) -> str:
        # Initialize the LLM (gpt-4o-mini) with your OpenAI API key
        openai.api_key = "key"
        return generate_response(prompt, retrieved_docs)

    @property
    def _identifying_params(self) -> dict:
        return {"model": "gpt-4o-mini"}

    @property
    def _llm_type(self) -> str:
        return "custom"

# Initializing the Gpt LLM and the RAG model
openaillm = GptLLM()
rag_model = RetrievalQA.from_chain_type(llm=openaillm, retriever=retriever)

# Example
query = "Explain the importance of Superposition from the context provided."
response_rag = rag_model.run(query)
print(f"RAG Response: {response_rag}")

C:\Users\r.vignesh\AppData\Local\Temp\ipykernel_8836\2561254141.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_rag = rag_model.run(query)


RAG Response: Superposition is a fundamental concept in quantum computing, as described in the context provided. It allows qubits, the building blocks of quantum computers, to exist in multiple states simultaneously. Unlike classical bits, which can only be in one of two states (0 or 1), qubits can represent both 0 and 1 at the same time due to superposition. This ability to be in multiple states enhances the computational power of quantum computers, enabling them to process a vast amount of information simultaneously.

Superposition is crucial for the functioning of quantum algorithms, such as those utilizing the Grover Operator, which systematically eliminates incorrect possibilities and amplifies the probability of the correct answer. By leveraging superposition, quantum computers can perform complex calculations more efficiently than classical computers, making it a key aspect of their potential to solve problems that are currently intractable. Overall, superposition plays a vital 

# Evaluation and Iteration
- **ROUGE Evaluation**: We used ROUGE scores to evaluate how well the generated answers from both the GPT-4o-mini model and the RAG model align with the reference answer. ROUGE is a common metric to measure the overlap between generated and reference text in natural language processing tasks.


In [13]:

# Calculating ROUGE scores for evaluation
def evaluate_rouge(prediction, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, prediction)
    return scores

# Example
reference_answer = "During the spin of a coin, there are 2 possibilities, both head and a tail. That position of a coin having both at the same time is what a qubit is. This state is called Superposition."

# Evaluating RAG-generated answer using ROUGE
rouge_scores_rag = evaluate_rouge(response_rag, reference_answer)
print("ROUGE Scores RAG:", rouge_scores_rag)

ROUGE Scores RAG: {'rouge1': Score(precision=0.14193548387096774, recall=0.6111111111111112, fmeasure=0.23036649214659685), 'rougeL': Score(precision=0.07741935483870968, recall=0.3333333333333333, fmeasure=0.1256544502617801)}


# Advanced Topics - Streamlit run
- **Caching**: We implemented a Python dictionary-based caching system to store the results of previously processed queries, improving performance for repeated queries.
- **Streamlit Interface**: We created a Streamlit-based interface to allow users to interact with the RAG model in real-time by inputting their queries and seeing the model's responses.


In [19]:
# Initialize the RAG model using Langchain's RetrievalQA with custom LLM
rag_model = RetrievalQA.from_chain_type(llm=openaillm, retriever=retriever)

In [20]:
# Streamlit-based interface for querying the RAG model
st.title("RAG Model Interface")

query = st.text_input("Enter your question:")
if query:
    response_rag = rag_model.run(query)
    st.write(f"Response: {response_rag}")

2024-10-17 00:59:41.985 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 00:59:41.986 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 00:59:41.988 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 00:59:41.989 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 00:59:41.989 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 00:59:41.990 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 00:59:41.991 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 00:59:41.992 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar